In [1]:
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset

import torch
import matplotlib.pyplot as plt

import os
from glob import glob
import numpy as np

from monai.inferers import sliding_window_inference

C:\Users\HP\anaconda3\lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
device = torch.device("cuda:0")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256), 
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

In [8]:
model_dir = 'C:/Users/HP/Desktop/Bitirme/TrSonuclar'

In [5]:
import torch
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadImage,
    EnsureChannelFirst,
    ScaleIntensity,
    Resize,
)
from monai.networks.blocks import ResidualUnit, Convolution

# Özel UNet modeli oluştur
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)

# Model ağırlıklarını yükle
weights = torch.load("C:/Users/HP/Desktop/Bitirme/TrSonuclar/best_metric_model.pth", weights_only=True)
model.load_state_dict(weights, strict=False)
model.eval()

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Se

In [426]:
def predict(image_path, model, device="cpu"):
    # Transform pipeline
    transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
        Resize((128, 128, 64))  # Model giriş boyutuna göre ayarlayın
    ])
    
    # Görüntüyü hazırla
    input_image = transforms(image_path)
    input_tensor = input_image.unsqueeze(0).to(device)
    
    # Tahmin yap
    with torch.no_grad():
        prediction = model(input_tensor)
        output = torch.argmax(prediction, dim=1)
    
    return output

# Örnek kullanım
image_path = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"
result = predict(image_path, model)

In [11]:
import nibabel as nib

In [15]:
image_path = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"
nifti_image = nib.load(image_path)

In [17]:
image_data = nifti_image.get_fdata()

In [21]:
image_data.shape

(512, 512, 65)

In [23]:
image_tensor = torch.tensor(image_data).float().unsqueeze(0).unsqueeze(0)

In [25]:
image_tensor.shape

torch.Size([1, 1, 512, 512, 65])

In [584]:
device = torch.device("cpu")

In [29]:
model = model.to(device)

In [31]:
test_outputs = sliding_window_inference(image_tensor.to(device), (128, 128, 64), 4, model)

In [33]:
test_outputs.shape

torch.Size([1, 2, 512, 512, 65])

In [556]:

# Boyutu [2, 512, 512, 65] olacak şekilde squeeze et
tensor_squeezed = test_outputs.squeeze(0)

# Tensorü iki [512, 512, 65] boyutunda ayır
tensor_split = torch.chunk(tensor_squeezed, 2, dim=0)

# Sonuçları yazdır
for i, t in enumerate(tensor_split):
    print(f"Tensor {i + 1} boyutu:", t.shape)


Tensor 1 boyutu: torch.Size([1, 512, 512, 65])
Tensor 2 boyutu: torch.Size([1, 512, 512, 65])


In [566]:
ikinci = tensor_split[1].squeeze(0)

In [564]:
ilk.shape

torch.Size([512, 512, 65])

In [35]:
sigmoid_activation = Activations(sigmoid=True)
test_outputs = sigmoid_activation(test_outputs)
test_outputs = test_outputs > 0.53

In [468]:
result.shape

torch.Size([1, 128, 128, 64])

In [470]:
tensor_numpy = result.squeeze(0)

In [472]:
tensor_numpy.shape

torch.Size([128, 128, 64])

In [373]:
sigmoid_activation = Activations(sigmoid=True)

In [504]:
#test_outputs = sigmoid_activation(tensor_numpy)
test_outputs = np.where(tensor_numpy > 0, 1, 0)

In [512]:
unique_elements, counts = np.unique(test_outputs, return_counts=True)

In [516]:
unique_elements

array([0, 1])

In [514]:
counts

array([1048557,      19], dtype=int64)

In [506]:
np.set_printoptions(threshold=np.inf)
print(test_outputs)

[[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [518]:
tensor_numpy = test_outputs.repeat_interleave(4, dim=0).repeat_interleave(4, dim=1)

AttributeError: 'numpy.ndarray' object has no attribute 'repeat_interleave'

In [458]:
tensor_numpy.shape

torch.Size([512, 512, 64])

In [387]:
new_channel = torch.zeros(512, 512, 1)

In [460]:
tensor_numpy = torch.cat((tensor_numpy, new_channel), dim=2)
tensor_numpy = tensor_numpy.cpu().numpy()
tensor_numpy = tensor_numpy.astype(np.float32)

In [361]:
np.set_printoptions(threshold=np.inf)
print(tensor_numpy)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [462]:
tensor_numpy.shape

(512, 512, 65)

In [37]:
test_outputs.shape

torch.Size([1, 2, 512, 512, 65])

In [41]:
segment = np.where(test_outputs[0,1,:,:,:], 1, 0)

In [578]:
test_outputs = np.where(ikinci, 1, 0)

In [43]:
nifti_image = nib.Nifti1Image( segment , affine=np.eye(4))

In [45]:
nifti_image

In [47]:
nib.save(nifti_image,'output_fileee.nii')

In [239]:
result

metatensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         ...,

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0,

In [151]:
import torch
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadImage,
    EnsureChannelFirst,
    ScaleIntensity,
    Resize,
)

def prepare_input(image_path, spatial_size=(96, 96, 96)):
    # Görüntüyü yüklemek ve ön işlemek için transform pipeline
    transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
        Resize(spatial_size=spatial_size)
    ])
    
    # Görüntüyü hazırla
    input_image = transforms(image_path)
    # Batch boyutunu ekle
    input_tensor = input_image.unsqueeze(0)
    return input_tensor

def predict(model, image_path, device="cuda" if torch.cuda.is_available() else "cpu"):
    # Modeli değerlendirme moduna al
    model.eval()
    model.to(device)
    
    # Görüntüyü hazırla
    input_tensor = prepare_input(image_path)
    input_tensor = input_tensor.to(device)
    
    # Tahmin yap
    with torch.no_grad():
        prediction = model(input_tensor)
    
    # Tahmin sonucunu işle (örnek olarak)
    output = torch.argmax(prediction, dim=1)
    
    return output


def load_model(model_path, device="cuda" if torch.cuda.is_available() else "cpu"):
    # UNet modelini oluştur
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=2,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
    )
    
    # Model ağırlıklarını yükle
    state_dict = torch.load(model_path, weights_only=True)
    model.load_state_dict(state_dict)
    model.to(device)
    return model


# Kullanım örneği:
image_path = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"  # görüntü yolu
model_path = "C:/Users/HP/Desktop/Bitirme/TrSonuclar/best_metric_model.pth"  # kaydedilmiş model yolu


model = load_model(model_path)
model.eval()
# Modeli yükle


# Tahmin yap
prediction = predict(model, image_path)

RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "model.0.conv.weight", "model.0.conv.bias", "model.0.adn.A.weight", "model.1.submodule.0.conv.weight", "model.1.submodule.0.conv.bias", "model.1.submodule.0.adn.A.weight", "model.1.submodule.1.submodule.0.conv.weight", "model.1.submodule.1.submodule.0.conv.bias", "model.1.submodule.1.submodule.0.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.0.conv.weight", "model.1.submodule.1.submodule.1.submodule.0.conv.bias", "model.1.submodule.1.submodule.1.submodule.0.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.conv.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.conv.bias", "model.1.submodule.1.submodule.1.submodule.1.submodule.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.2.conv.weight", "model.1.submodule.1.submodule.1.submodule.2.conv.bias", "model.1.submodule.1.submodule.1.submodule.2.adn.A.weight", "model.1.submodule.1.submodule.2.conv.weight

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

In [16]:
test_transforms = Compose(
    [
        LoadImaged(keys=["vol"]),
        AddChanneld(keys=["vol"]),
        Spacingd(keys=["vol"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
        Orientationd(keys=["vol"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
        CropForegroundd(keys=['vol'], source_key='vol'),
        Resized(keys=["vol"], spatial_size=[128,128,64]),   
        ToTensord(keys=["vol", "seg"]),
    ]
)

In [56]:
test_ds = CacheDataset(data="C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii", cache_rate=1.0)
test_loader = DataLoader(test_ds, batch_size=1)

Loading dataset: 100%|██████████| 61/61 [00:00<?, ?it/s]


In [62]:
# DataLoader'dan bir batch al
test_patient = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"

# test_patient'in doğru formatta olup olmadığını kontrol edin
print("test_patient:", test_patient)

# Eğer test_patient bir liste ise ve dosya yolları içeriyorsa:
if isinstance(test_patient, list):
    t_volume_path = test_patient[0]  # İlk dosya yolunu al
    print("Volume path:", t_volume_path)
    
    # Yolu doğrulayın
    if not os.path.exists(t_volume_path):
        raise FileNotFoundError(f"Dosya yolu bulunamadı: {t_volume_path}")

    # Veriyi yükleyin
    t_volume = load_transform(t_volume_path).to(device)
else:
    raise ValueError("test_patient beklenmeyen bir format içeriyor!")


test_patient: C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii


ValueError: test_patient beklenmeyen bir format içeriyor!

In [78]:
print(t_volume.shape)


torch.Size([1, 512, 512, 65])


In [96]:
# Bu durumda sadece ilk iki boyut olan height ve width üzerinde işlem yapabilirsiniz:
t_volume_single_channel = t_volume[0, :, :, :]  # [height, width, depth]


ValueError: Sequence must have length 1, got 2.

In [92]:
# Sliding Window Inference işlemi:
test_outputs = sliding_window_inference(t_volume_single_channel.unsqueeze(0), roi_size_2d, sw_batch_size, model)


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3, 3], expected input[1, 4, 512, 128, 128] to have 1 channels, but got 4 channels instead

In [98]:
# Aktivasyon ve eşikleme işlemleri

sigmoid_activation = Activations(sigmoid=True)
test_outputs = sigmoid_activation(test_outputs)
test_outputs = test_outputs > 0.53  # Eşikleme (Thresholding)

# Görselleştirme
for i in range(t_volume_single_channel.shape[2]):  # depth üzerinde iterasyon yapar
    plt.figure("check", (18, 6))
    
    # Orijinal görüntü
    plt.subplot(1, 3, 1)
    plt.title(f"image {i}")
    plt.imshow(t_volume_single_channel[:, :, i].cpu(), cmap="gray")
    
    # Çıktı görüntüsü
    plt.subplot(1, 3, 2)
    plt.title(f"output {i}")
    plt.imshow(t_volume_single_channel[:, :, i].cpu(), cmap="gray")
    plt.imshow(test_outputs.detach().cpu()[0, 0, :, :, i], alpha=0.5)
    plt.show()


NameError: name 'test_outputs' is not defined

In [118]:
from monai.transforms import LoadImage, EnsureChannelFirst, ScaleIntensityRange, ToTensor
from monai.transforms import Compose
import torch

# Transforms tanımlama
load_transform = Compose([
    LoadImage(image_only=True),
    EnsureChannelFirst(),
    ScaleIntensityRange(a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
    ToTensor()
])

sw_batch_size = 4
roi_size = (128, 128, 64)

with torch.no_grad():
    # DataLoader'dan ilk batch'i al
    test_patient = next(iter(test_loader))  # Bu sadece dosya yollarını döner
    t_volume_path = test_patient[0]  # İlk dosya yolunu al
    t_volume = load_transform(t_volume_path).to(device)  # Veriyi yükle ve işleyip cihaza taşı
    
    # Sliding window inference
    test_outputs = sliding_window_inference(t_volume, roi_size, sw_batch_size, model)
    
    # Aktivasyon ve eşikleme işlemleri
    sigmoid_activation = Activations(sigmoid=True)
    test_outputs = sigmoid_activation(test_outputs)
    test_outputs = test_outputs > 0.53  # Eşikleme (Thresholding)
    
    # Dilim görselleştirme
    for i in range(64):
        plt.figure("check", (18, 6))
        
        # Orijinal görüntü
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(t_volume[0, :, :, i].cpu(), cmap="gray")
        
        # Çıktı görüntüsü
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(t_volume[0, :, :, i].cpu(), cmap="gray")  # Orijinal
        plt.imshow(test_outputs.detach().cpu()[0, 0, :, :, i], alpha=0.5)  # Model çıktısı
        plt.show()


RuntimeError: applying transform <monai.transforms.io.array.LoadImage object at 0x0000021B8A423B20>

In [116]:
test_patient = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"

In [120]:
# Add this debug print before trying to load the volume
print("Image path:", t_volume_path)
test_patient = next(iter(test_loader))
t_volume_path = test_patient[0]
print("Type of path:", type(t_volume_path))
print(test_patient)

Image path: C
Type of path: <class 'str'>
['C']


In [140]:
import torch
import torch.nn as nn
from monai.transforms import LoadImage, EnsureChannelFirst, ScaleIntensityRange, ToTensor, Compose
from monai.inferers import sliding_window_inference
import matplotlib.pyplot as plt
import os

# Transforms tanımlama
load_transform = Compose([
    LoadImage(image_only=True),
    EnsureChannelFirst(),
    ScaleIntensityRange(a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
    ToTensor()
])

# Test görselinin yolu
test_image_path = "C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii"

# Parametreler
sw_batch_size = 4
roi_size = (128, 128, 32)  # 3D roi_size

# Görüntüyü yükle
print(f"Loading image from: {test_image_path}")
t_volume = load_transform(test_image_path).to(device)
print("Original image shape:", t_volume.shape)

# Kanal sayısını düzelt (4'ten 1'e)
if t_volume.shape[1] != 1:
    # Ortalama alarak tek kanala düşür
    t_volume = t_volume.mean(dim=1, keepdim=True)
print("Adjusted image shape:", t_volume.shape)

with torch.no_grad():
    # Sliding window inference
    test_outputs = sliding_window_inference(
        inputs=t_volume,
        roi_size=roi_size,
        sw_batch_size=sw_batch_size,
        predictor=model,
        overlap=0.5
    )
    
    # Sigmoid aktivasyonu ve eşikleme
    sigmoid = nn.Sigmoid()
    test_outputs = sigmoid(test_outputs)
    test_outputs = test_outputs > 0.53
    
    # Görselleştirme
    for i in range(t_volume.shape[-1]):
        plt.figure("check", (18, 6))
        
        # Orijinal görüntü
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(t_volume[0, 0, :, :, i].cpu(), cmap="gray")
        
        # Çıktı görüntüsü
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(t_volume[0, 0, :, :, i].cpu(), cmap="gray")
        plt.imshow(test_outputs[0, 0, :, :, i].cpu(), alpha=0.5)
        plt.show()

Loading image from: C:/Users/HP/Desktop/Bitirme/TrNii/TestVolumes/colon_005_0.nii
Original image shape: torch.Size([1, 512, 512, 65])
Adjusted image shape: torch.Size([1, 1, 512, 65])


ValueError: Sequence must have length 2, got 3.

In [128]:
!pip install monai --upgrade

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.1 MB/s eta 0:00:00
  Attempting uninstall: monai
    Found existing installation: monai 1.2.0
    Uninstalling monai-1.2.0:
      Successfully uninstalled monai-1.2.0
